|Created bounding box

In [1]:
geojson_geometry = {
    "type": "Polygon",
    "coordinates":  [
        [
            [-47.42813554632008,-22.451060725842694],
            [-47.42813554632008,-22.477697748323862],
            [-47.410670583298526,-22.477697748323862],
            [-47.410670583298526,-22.451060725842694],
            [-47.42813554632008,-22.451060725842694]
        ]
    ]
}

In [1]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

KeyboardInterrupt: 

In [ ]:
# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2017-08-31T00:00:00.000Z",
    "lte": "2017-09-01T00:00:00.000Z"
  }
}

In [ ]:
# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

In [ ]:
# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:

import os

# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PLAKcaa4ce6cd0464a058d71353bf2bca50b'

In [ ]:
import json
import requests
from requests.auth import HTTPBasicAuth

item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

geojson = search_result.json()

# let's look at the first result
print(list(geojson.items())[1][1][0])

{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene/items/20170831_123632_0f35', 'assets': 'https://api.planet.com/data/v1/item-types/PSScene/items/20170831_123632_0f35/assets/', 'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene/items/20170831_123632_0f35/thumb'}, '_permissions': [], 'assets': ['basic_analytic_4b', 'basic_analytic_4b_rpc', 'basic_analytic_4b_xml', 'basic_udm2', 'ortho_analytic_3b', 'ortho_analytic_3b_xml', 'ortho_analytic_4b', 'ortho_analytic_4b_sr', 'ortho_analytic_4b_xml', 'ortho_udm2', 'ortho_visual'], 'geometry': {'coordinates': [[[-47.47615244795111, -22.372967593987035], [-47.492771651772884, -22.44800112845778], [-47.24280931989658, -22.49495353375397], [-47.22674507870009, -22.419647095359334], [-47.47615244795111, -22.372967593987035]]], 'type': 'Polygon'}, 'id': '20170831_123632_0f35', 'properties': {'acquired': '2017-08-31T12:36:32.403774Z', 'anomalous_pixels': 0.01, 'clear_confidence_percent': 97, 'clear_percent': 98, 'cl

In [ ]:
# let's look at the first result
print(json.dumps(search_result.json(), indent=1))

{
 "_links": {
  "_first": "https://api.planet.com/data/v1/searches/db5346baf9864e1db7eb8d82202a6752/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D",
  "_next": "https://api.planet.com/data/v1/searches/db5346baf9864e1db7eb8d82202a6752/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIxLTAyLTE0VDE1OjI3OjQ1LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTcwODMxXzEyMzYzM18wZjM1IiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319",
  "_self": "https://api.planet.com/data/v1/searches/db5346baf9864e1db7eb8d82202a6752/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D"
 },
 "features": [
  {
   "_links": {
    "

In [ ]:
# extract image IDs only
image_ids = [feature['id'] for feature in geojson['features']]
print(image_ids)

['20170831_123632_0f35', '20170831_123633_0f35']


In [ ]:
# For demo purposes, just grab the first image ID
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )

# List of asset types available for this particular satellite image
print(result.json().keys())

dict_keys([])


In [ ]:
# This is "inactive" if the "ortho_analytic_4b" asset has not yet been activated; otherwise 'active'
print(result.json()['ortho_analytic_4b']['status'])

KeyError: 'ortho_analytic_4b'